In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from numpy.ma.core import mean
import math 
import matplotlib.pyplot as plt
import random


In [4]:
recipes = pd.read_csv("RAW_recipes.csv",encoding="Latin1")
ratings = pd.read_csv("interactions_train.csv")
ratings_v1=ratings.copy()
ratings_v1.drop(['rating'], axis=1, inplace=True)

In [5]:
unique_users=ratings_v1['user_id'].unique()

In [6]:
users_diseases = pd.DataFrame()
users_diseases['users']=pd.Series(unique_users)


In [7]:
users_diseases['Cancer']=False
users_diseases['Obesity']=False
users_diseases['Diabetes']=False
users_diseases['Cardiovascular']=False
users_diseases['Dental']=False
users_diseases['Osteoporosis']=False
users_diseases

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
0,2046,False,False,False,False,False,False
1,1773,False,False,False,False,False,False
2,2312,False,False,False,False,False,False
3,2625,False,False,False,False,False,False
4,2999,False,False,False,False,False,False
...,...,...,...,...,...,...,...
25071,2001355273,False,False,False,False,False,False
25072,2002025577,False,False,False,False,False,False
25073,2001773389,False,False,False,False,False,False
25074,2001688000,False,False,False,False,False,False


In [8]:
healthy_users=users_diseases.sample(frac=0.4, random_state=1)
healthy_users.drop_duplicates(['users'], keep='last')
users_diseases=users_diseases[~ users_diseases['users'].isin(healthy_users['users'])]

In [9]:
users_diseases

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
1,1773,False,False,False,False,False,False
3,2625,False,False,False,False,False,False
8,5523,False,False,False,False,False,False
9,42189,False,False,False,False,False,False
10,5929,False,False,False,False,False,False
...,...,...,...,...,...,...,...
25069,2001993143,False,False,False,False,False,False
25072,2002025577,False,False,False,False,False,False
25073,2001773389,False,False,False,False,False,False
25074,2001688000,False,False,False,False,False,False


In [10]:
cancers_gen1=users_diseases.sample(frac=0.08, random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen1['Cancer']=True

cancers_gen2=cancers_gen1.sample(frac=0.312, random_state=1).drop_duplicates(['users'], keep='last')

cancers_gen3=cancers_gen2.sample(frac=0.2, random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen3['Obesity']=True

cancers_gen4=cancers_gen2[~ cancers_gen2['users'].isin(cancers_gen3['users'])].drop_duplicates(['users'], keep='last')
cancers_gen4=cancers_gen4.sample(frac=0.15, random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen4['Diabetes']=True

cancers_gen5=cancers_gen2[~ cancers_gen2['users'].isin(cancers_gen4['users'])& ~ cancers_gen2['users'].isin(cancers_gen3['users'])].drop_duplicates(['users'], keep='last')
cancers_gen5=cancers_gen5.sample(frac=0.2,  random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen5['Cardiovascular']=True

cancers_gen6=cancers_gen2[~ cancers_gen2['users'].isin(cancers_gen4['users'])& ~ cancers_gen2['users'].isin(cancers_gen3['users'])& ~ cancers_gen2['users'].isin(cancers_gen5['users'])].drop_duplicates(['users'], keep='last')
cancers_gen6=cancers_gen6.sample(frac=0.3, random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen6['Osteoporosis']=True

cancers_gen7=cancers_gen2[~ cancers_gen2['users'].isin(cancers_gen4['users'])& ~ cancers_gen2['users'].isin(cancers_gen3['users'])& ~ cancers_gen2['users'].isin(cancers_gen5['users'])& ~ cancers_gen2['users'].isin(cancers_gen6['users'])].drop_duplicates(['users'], keep='last')
cancers_gen7=cancers_gen7.sample(frac=0.15,  random_state=1).drop_duplicates(['users'], keep='last')
cancers_gen7['Dental']=True

In [11]:
cancer_with_other=pd.concat([cancers_gen3,cancers_gen4,cancers_gen5,cancers_gen6,cancers_gen7], axis=0).drop_duplicates()
tmp=cancers_gen1[~ cancers_gen1['users'].isin(cancer_with_other['users'])].drop_duplicates()
cancer_with_other

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
15686,487703,True,True,False,False,False,False
9454,336834,True,True,False,False,False,False
16953,359834,True,True,False,False,False,False
4967,192554,True,True,False,False,False,False
12739,591575,True,True,False,False,False,False
...,...,...,...,...,...,...,...
8348,334126,True,False,False,False,True,False
23861,2567666,True,False,False,False,True,False
19303,709476,True,False,False,False,True,False
1839,68605,True,False,False,False,True,False


In [12]:
cancer_with_2plus=tmp.sample(frac=0.245, random_state=1).drop_duplicates(['users'], keep='last')

cancer_with_2plus_1=cancer_with_2plus.sample(frac=0.208).drop_duplicates(['users'], keep='last')
cancer_with_2plus_1['Obesity']=True
cancer_with_2plus_1['Cardiovascular']=True

cancer_with_2plus_2=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_2=cancer_with_2plus_2.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_2['Dental']=True
cancer_with_2plus_2['Osteoporosis']=True

cancer_with_2plus_3=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_3=cancer_with_2plus_3.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_3['Cardiovascular']=True
cancer_with_2plus_3['Diabetes']=True

cancer_with_2plus_4=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_4=cancer_with_2plus_4.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_4['Diabetes']=True
cancer_with_2plus_4['Dental']=True

cancer_with_2plus_5=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_5=cancer_with_2plus_5.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_5['Osteoporosis']=True
cancer_with_2plus_5['Diabetes']=True

cancer_with_2plus_6=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_5['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_6=cancer_with_2plus_6.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_6['Osteoporosis']=True
cancer_with_2plus_6['Obesity']=True

cancer_with_2plus_7=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_5['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_6['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_7=cancer_with_2plus_7.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_7['Osteoporosis']=True
cancer_with_2plus_7['Cardiovascular']=True

cancer_with_2plus_8=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_5['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_6['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_7['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_8=cancer_with_2plus_8.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_8['Obesity']=True
cancer_with_2plus_8['Dental']=True

cancer_with_2plus_9=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_5['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_6['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_7['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_8['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_9=cancer_with_2plus_9.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_9['Cardiovascular']=True
cancer_with_2plus_9['Dental']=True

cancer_with_2plus_10=cancer_with_2plus[~ cancer_with_2plus['users'].isin(cancer_with_2plus_1['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_2['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_3['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_4['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_5['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_6['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_7['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_8['users'])& ~ cancer_with_2plus['users'].isin(cancer_with_2plus_9['users'])].drop_duplicates(['users'], keep='last')
cancer_with_2plus_10=cancer_with_2plus_10.sample(frac=0.088).drop_duplicates(['users'], keep='last')
cancer_with_2plus_10['Obesity']=True
cancer_with_2plus_10['Diabetes']=True


In [13]:
cancer_with_2plus=pd.concat([cancer_with_2plus_1,cancer_with_2plus_2,cancer_with_2plus_3,cancer_with_2plus_4,cancer_with_2plus_5,cancer_with_2plus_6,cancer_with_2plus_7,cancer_with_2plus_8,cancer_with_2plus_9,cancer_with_2plus_10], axis=0).drop_duplicates()
cancer_with_2plus

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
7000,40035,True,True,False,True,False,False
2020,86604,True,True,False,True,False,False
19236,302867,True,True,False,True,False,False
21207,1516651,True,True,False,True,False,False
355,30674,True,True,False,True,False,False
...,...,...,...,...,...,...,...
23088,1383525,True,True,True,False,False,False
367,26512,True,True,True,False,False,False
22824,798690,True,True,True,False,False,False
5089,193489,True,True,True,False,False,False


In [14]:
only_cancer=cancers_gen1[~ cancers_gen1['users'].isin(cancer_with_2plus['users'])& ~ cancers_gen1['users'].isin(cancer_with_other['users'])].drop_duplicates()
only_cancer


,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
17639,575940,True,False,False,False,False,False
7273,232086,True,False,False,False,False,False
7382,291378,True,False,False,False,False,False
20708,1028637,True,False,False,False,False,False
4054,118268,True,False,False,False,False,False
...,...,...,...,...,...,...,...
24664,2000204972,True,False,False,False,False,False
3758,146310,True,False,False,False,False,False
14955,616362,True,False,False,False,False,False
24221,2789235,True,False,False,False,False,False


In [15]:
cancer_dataset=pd.concat([only_cancer,cancer_with_other,cancer_with_2plus]).drop_duplicates()
cancer_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
17639,575940,True,False,False,False,False,False
7273,232086,True,False,False,False,False,False
7382,291378,True,False,False,False,False,False
20708,1028637,True,False,False,False,False,False
4054,118268,True,False,False,False,False,False
...,...,...,...,...,...,...,...
23088,1383525,True,True,True,False,False,False
367,26512,True,True,True,False,False,False
22824,798690,True,True,True,False,False,False
5089,193489,True,True,True,False,False,False


In [16]:
diabetes_gen=users_diseases[~ users_diseases['users'].isin(cancer_dataset['users'])]
diabetes_gen1=diabetes_gen.sample(frac=0.16, random_state=1).drop_duplicates(['users'], keep='last')
diabetes_gen1['Diabetes']=True

diabetes_gen2=diabetes_gen1.sample(frac=0.293, random_state=1).drop_duplicates(['users'], keep='last')

diabetes_gen3=diabetes_gen2.sample(frac=0.5, random_state=1).drop_duplicates(['users'], keep='last')
diabetes_gen3['Obesity']=True

diabetes_gen4=diabetes_gen2[~ diabetes_gen2['users'].isin(diabetes_gen3['users'])].drop_duplicates(['users'], keep='last')
diabetes_gen4=diabetes_gen4.sample(frac=0.12, random_state=1).drop_duplicates(['users'], keep='last')
diabetes_gen4['Dental']=True

diabetes_gen5=diabetes_gen2[~ diabetes_gen2['users'].isin(diabetes_gen4['users'])& ~ diabetes_gen2['users'].isin(diabetes_gen3['users'])].drop_duplicates(['users'], keep='last')
diabetes_gen5=diabetes_gen5.sample(frac=0.2,  random_state=1).drop_duplicates(['users'], keep='last')
diabetes_gen5['Cardiovascular']=True

diabetes_gen6=diabetes_gen2[~ diabetes_gen2['users'].isin(diabetes_gen4['users'])& ~ diabetes_gen2['users'].isin(diabetes_gen3['users'])& ~ diabetes_gen2['users'].isin(diabetes_gen5['users'])].drop_duplicates(['users'], keep='last')
diabetes_gen6=diabetes_gen6.sample(frac=0.18, random_state=1).drop_duplicates(['users'], keep='last')
diabetes_gen6['Osteoporosis']=True

In [17]:
diabetes_with_other=pd.concat([diabetes_gen3,diabetes_gen4,diabetes_gen5,diabetes_gen6], axis=0).drop_duplicates()
tmp=diabetes_gen1[~ diabetes_gen1['users'].isin(diabetes_with_other['users'])].drop_duplicates()
diabetes_with_other

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
19319,1222615,False,True,True,False,False,False
20099,744267,False,True,True,False,False,False
7702,285771,False,True,True,False,False,False
2922,117776,False,True,True,False,False,False
22661,1580907,False,True,True,False,False,False
...,...,...,...,...,...,...,...
21245,1497621,False,False,True,False,False,True
22767,1823125,False,False,True,False,False,True
11594,308490,False,False,True,False,False,True
15636,688994,False,False,True,False,False,True


In [18]:
diabetes_gen1

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
8450,313335,False,False,True,False,False,False
22354,228801,False,False,True,False,False,False
22721,1716057,False,False,True,False,False,False
13088,314769,False,False,True,False,False,False
16516,913578,False,False,True,False,False,False
...,...,...,...,...,...,...,...
9629,239901,False,False,True,False,False,False
8411,220433,False,False,True,False,False,False
24729,2000389212,False,False,True,False,False,False
16787,911706,False,False,True,False,False,False


In [19]:
diabetes_with_2plus=tmp.sample(frac=0.235, random_state=1).drop_duplicates(['users'], keep='last')

diabetes_with_2plus_1=diabetes_with_2plus.sample(frac=0.35).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_1['Obesity']=True
diabetes_with_2plus_1['Cardiovascular']=True

diabetes_with_2plus_2=diabetes_with_2plus[~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_1['users'])].drop_duplicates(['users'], keep='last')
diabetes_with_2plus_2=diabetes_with_2plus_2.sample(frac=0.13).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_2['Dental']=True
diabetes_with_2plus_2['Osteoporosis']=True


diabetes_with_2plus_3=diabetes_with_2plus[~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_1['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_2['users'])].drop_duplicates(['users'], keep='last')
diabetes_with_2plus_3=diabetes_with_2plus_3.sample(frac=0.13).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_3['Osteoporosis']=True
diabetes_with_2plus_3['Obesity']=True

diabetes_with_2plus_4=diabetes_with_2plus[~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_1['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_2['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_3['users'])].drop_duplicates(['users'], keep='last')
diabetes_with_2plus_4=diabetes_with_2plus_4.sample(frac=0.13).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_4['Osteoporosis']=True
diabetes_with_2plus_4['Cardiovascular']=True

diabetes_with_2plus_5=diabetes_with_2plus[~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_1['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_2['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_3['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_4['users'])].drop_duplicates(['users'], keep='last')
diabetes_with_2plus_5=diabetes_with_2plus_5.sample(frac=0.13).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_5['Obesity']=True
diabetes_with_2plus_5['Dental']=True

diabetes_with_2plus_6=diabetes_with_2plus[~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_1['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_2['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_3['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_4['users'])& ~ diabetes_with_2plus['users'].isin(diabetes_with_2plus_5['users'])].drop_duplicates(['users'], keep='last')
diabetes_with_2plus_6=diabetes_with_2plus_6.sample(frac=0.13).drop_duplicates(['users'], keep='last')
diabetes_with_2plus_6['Cardiovascular']=True
diabetes_with_2plus_6['Dental']=True


In [20]:
diabetes_with_2plus=pd.concat([diabetes_with_2plus_1,diabetes_with_2plus_2,diabetes_with_2plus_3,diabetes_with_2plus_4,diabetes_with_2plus_5,diabetes_with_2plus_6], axis=0).drop_duplicates()
diabetes_with_2plus

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
8118,327115,False,True,True,True,False,False
13388,246695,False,True,True,True,False,False
17660,970032,False,True,True,True,False,False
9659,389683,False,True,True,True,False,False
13448,649116,False,True,True,True,False,False
...,...,...,...,...,...,...,...
8779,296790,False,False,True,True,True,False
11267,93438,False,False,True,True,True,False
5887,201392,False,False,True,True,True,False
7109,281399,False,False,True,True,True,False


In [21]:
only_diabetes=diabetes_gen1[~ diabetes_gen1['users'].isin(diabetes_with_2plus['users'])& ~ diabetes_gen1['users'].isin(diabetes_with_other['users'])].drop_duplicates()
only_diabetes


,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
22354,228801,False,False,True,False,False,False
22721,1716057,False,False,True,False,False,False
16516,913578,False,False,True,False,False,False
4951,195212,False,False,True,False,False,False
18650,1169153,False,False,True,False,False,False
...,...,...,...,...,...,...,...
21657,53495,False,False,True,False,False,False
9629,239901,False,False,True,False,False,False
8411,220433,False,False,True,False,False,False
24729,2000389212,False,False,True,False,False,False


In [22]:
diabetes_dataset=pd.concat([only_diabetes,diabetes_with_other,diabetes_with_2plus]).drop_duplicates()
diabetes_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
22354,228801,False,False,True,False,False,False
22721,1716057,False,False,True,False,False,False
16516,913578,False,False,True,False,False,False
4951,195212,False,False,True,False,False,False
18650,1169153,False,False,True,False,False,False
...,...,...,...,...,...,...,...
8779,296790,False,False,True,True,True,False
11267,93438,False,False,True,True,True,False
5887,201392,False,False,True,True,True,False
7109,281399,False,False,True,True,True,False


In [23]:
heart_gen=users_diseases[~ users_diseases['users'].isin(cancer_dataset['users'])& ~ users_diseases['users'].isin(diabetes_dataset['users'])]
heart_gen1=heart_gen.sample(frac=0.112, random_state=1).drop_duplicates(['users'], keep='last')
heart_gen1['Cardiovascular']=True

heart_gen2=heart_gen1.sample(frac=0.361, random_state=1).drop_duplicates(['users'], keep='last')

heart_gen3=heart_gen2.sample(frac=0.6, random_state=1).drop_duplicates(['users'], keep='last')
heart_gen3['Obesity']=True

heart_gen4=heart_gen2[~ heart_gen2['users'].isin(heart_gen3['users'])].drop_duplicates(['users'], keep='last')
heart_gen4=heart_gen4.sample(frac=0.1, random_state=1).drop_duplicates(['users'], keep='last')
heart_gen4['Dental']=True

heart_gen5=heart_gen2[~ heart_gen2['users'].isin(heart_gen4['users'])& ~ heart_gen2['users'].isin(heart_gen3['users'])].drop_duplicates(['users'], keep='last')
heart_gen5=heart_gen5.sample(frac=0.3,  random_state=1).drop_duplicates(['users'], keep='last')
heart_gen5['Osteoporosis']=True


In [24]:
heart_with_other=pd.concat([heart_gen3,heart_gen4,heart_gen5], axis=0).drop_duplicates()
tmp=heart_gen1[~ heart_gen1['users'].isin(heart_with_other['users'])].drop_duplicates()
heart_with_other

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
21365,1415227,False,True,False,True,False,False
18144,1091792,False,True,False,True,False,False
20087,1323045,False,True,False,True,False,False
23998,2351513,False,True,False,True,False,False
1042,53077,False,True,False,True,False,False
...,...,...,...,...,...,...,...
22003,1601457,False,False,False,True,False,True
2026,15271,False,False,False,True,False,True
4716,186487,False,False,False,True,False,True
22069,1623783,False,False,False,True,False,True


In [25]:
heart_with_2plus=tmp.sample(frac=0.363, random_state=1).drop_duplicates(['users'], keep='last')

heart_with_2plus_1=heart_with_2plus.sample(frac=0.15).drop_duplicates(['users'], keep='last')
heart_with_2plus_1['Obesity']=True
heart_with_2plus_1['Dental']=True

heart_with_2plus_2=heart_with_2plus[~ heart_with_2plus['users'].isin(heart_with_2plus_1['users'])].drop_duplicates(['users'], keep='last')
heart_with_2plus_2=heart_with_2plus_2.sample(frac=0.15).drop_duplicates(['users'], keep='last')
heart_with_2plus_2['Dental']=True
heart_with_2plus_2['Osteoporosis']=True


heart_with_2plus_3=heart_with_2plus[~ heart_with_2plus['users'].isin(heart_with_2plus_1['users'])& ~ heart_with_2plus['users'].isin(heart_with_2plus_2['users'])].drop_duplicates(['users'], keep='last')
heart_with_2plus_3=heart_with_2plus_3.sample(frac=0.7).drop_duplicates(['users'], keep='last')
heart_with_2plus_3['Osteoporosis']=True
heart_with_2plus_3['Obesity']=True





In [26]:
heart_with_2plus=pd.concat([heart_with_2plus_1,heart_with_2plus_2,heart_with_2plus_3], axis=0).drop_duplicates()
heart_with_2plus

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
11489,240471,False,True,False,True,True,False
9131,229130,False,True,False,True,True,False
12768,590178,False,True,False,True,True,False
3778,108773,False,True,False,True,True,False
10839,471952,False,True,False,True,True,False
...,...,...,...,...,...,...,...
19332,347922,False,True,False,True,False,True
4270,168878,False,True,False,True,False,True
4772,21225,False,True,False,True,False,True
7604,273799,False,True,False,True,False,True


In [27]:
only_heart=heart_gen1[~ heart_gen1['users'].isin(heart_with_2plus['users'])& ~ heart_gen1['users'].isin(heart_with_other['users'])].drop_duplicates()
only_heart


,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
8707,233532,False,False,False,True,False,False
11359,381688,False,False,False,True,False,False
12313,184325,False,False,False,True,False,False
9203,375244,False,False,False,True,False,False
16248,298070,False,False,False,True,False,False
...,...,...,...,...,...,...,...
2945,75805,False,False,False,True,False,False
20990,1108916,False,False,False,True,False,False
16676,906006,False,False,False,True,False,False
22095,1649832,False,False,False,True,False,False


In [28]:
heart_dataset=pd.concat([only_heart,heart_with_other,heart_with_2plus]).drop_duplicates()
heart_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
8707,233532,False,False,False,True,False,False
11359,381688,False,False,False,True,False,False
12313,184325,False,False,False,True,False,False
9203,375244,False,False,False,True,False,False
16248,298070,False,False,False,True,False,False
...,...,...,...,...,...,...,...
19332,347922,False,True,False,True,False,True
4270,168878,False,True,False,True,False,True
4772,21225,False,True,False,True,False,True
7604,273799,False,True,False,True,False,True


In [29]:
ob_gen=users_diseases[~ users_diseases['users'].isin(cancer_dataset['users'])& ~ users_diseases['users'].isin(diabetes_dataset['users'])& ~ users_diseases['users'].isin(heart_dataset['users'])]
ob_gen1=ob_gen.sample(frac=0.208, random_state=1).drop_duplicates(['users'], keep='last')
ob_gen1['Obesity']=True

ob_gen2=ob_gen1.sample(frac=0.363, random_state=1).drop_duplicates(['users'], keep='last')

ob_gen3=ob_gen2.sample(frac=0.5, random_state=1).drop_duplicates(['users'], keep='last')
ob_gen3['Osteoporosis']=True

ob_gen4=ob_gen2[~ ob_gen2['users'].isin(ob_gen3['users'])].drop_duplicates(['users'], keep='last')
ob_gen4['Dental']=True


In [30]:
ob_with_other=pd.concat([ob_gen3,ob_gen4], axis=0).drop_duplicates()
tmp=ob_gen1[~ ob_gen1['users'].isin(ob_with_other['users'])].drop_duplicates()
ob_with_other

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
22817,1813203,False,True,False,False,False,True
3009,111263,False,True,False,False,False,True
12620,416738,False,True,False,False,False,True
19344,1250824,False,True,False,False,False,True
2233,62883,False,True,False,False,False,True
...,...,...,...,...,...,...,...
6419,236254,False,True,False,False,True,False
464,35166,False,True,False,False,True,False
10110,282843,False,True,False,False,True,False
15817,570444,False,True,False,False,True,False


In [31]:
ob_with_2plus=tmp.sample(frac=0.361, random_state=1).drop_duplicates(['users'], keep='last')

ob_with_2plus['Osteoporosis']=True
ob_with_2plus['Dental']=True


In [32]:
ob_with_2plus

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
16384,598603,False,True,False,False,True,True
17374,929782,False,True,False,False,True,True
4346,171163,False,True,False,False,True,True
4288,160577,False,True,False,False,True,True
12912,604671,False,True,False,False,True,True
...,...,...,...,...,...,...,...
20003,1322865,False,True,False,False,True,True
14288,712800,False,True,False,False,True,True
24156,2762963,False,True,False,False,True,True
13200,233500,False,True,False,False,True,True


In [33]:
only_ob=ob_gen1[~ ob_gen1['users'].isin(ob_with_other['users'])& ~ ob_gen1['users'].isin(ob_with_2plus['users'])].drop_duplicates()
only_ob

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
22240,1002864,False,True,False,False,False,False
7423,139381,False,True,False,False,False,False
17905,799503,False,True,False,False,False,False
6243,53959,False,True,False,False,False,False
15485,798459,False,True,False,False,False,False
...,...,...,...,...,...,...,...
2971,115048,False,True,False,False,False,False
17925,1080477,False,True,False,False,False,False
10548,391106,False,True,False,False,False,False
10957,475302,False,True,False,False,False,False


In [34]:
ob_dataset=pd.concat([only_ob,ob_with_other,ob_with_2plus]).drop_duplicates()
ob_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
22240,1002864,False,True,False,False,False,False
7423,139381,False,True,False,False,False,False
17905,799503,False,True,False,False,False,False
6243,53959,False,True,False,False,False,False
15485,798459,False,True,False,False,False,False
...,...,...,...,...,...,...,...
20003,1322865,False,True,False,False,True,True
14288,712800,False,True,False,False,True,True
24156,2762963,False,True,False,False,True,True
13200,233500,False,True,False,False,True,True


In [35]:
den_gen=users_diseases[~ users_diseases['users'].isin(cancer_dataset['users'])& ~ users_diseases['users'].isin(diabetes_dataset['users'])& ~ users_diseases['users'].isin(heart_dataset['users'])& ~ users_diseases['users'].isin(ob_dataset['users'])]
den_gen1=den_gen.sample(frac=0.224, random_state=1).drop_duplicates(['users'], keep='last')
den_gen1['Dental']=True

den_with_other=den_gen1.sample(frac=0.333, random_state=1).drop_duplicates(['users'], keep='last')

den_with_other['Osteoporosis']=True



In [36]:
tmp=ob_gen1[~ ob_gen1['users'].isin(den_with_other['users'])].drop_duplicates()
den_with_other

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
400,28408,False,False,False,False,True,True
1267,51437,False,False,False,False,True,True
24481,1519155,False,False,False,False,True,True
17486,633221,False,False,False,False,True,True
24244,903635,False,False,False,False,True,True
...,...,...,...,...,...,...,...
6642,139238,False,False,False,False,True,True
3391,121185,False,False,False,False,True,True
21403,193446,False,False,False,False,True,True
7332,244119,False,False,False,False,True,True


In [37]:
only_den=den_gen1[~ den_gen1['users'].isin(den_with_other['users'])].drop_duplicates()
only_den

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
5763,5842,False,False,False,False,True,False
19643,1193804,False,False,False,False,True,False
128,20076,False,False,False,False,True,False
11931,344342,False,False,False,False,True,False
9383,388463,False,False,False,False,True,False
...,...,...,...,...,...,...,...
229,26399,False,False,False,False,True,False
9236,366104,False,False,False,False,True,False
4545,180779,False,False,False,False,True,False
23270,1979510,False,False,False,False,True,False


In [38]:
den_dataset=pd.concat([only_den,den_with_other]).drop_duplicates()
den_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
5763,5842,False,False,False,False,True,False
19643,1193804,False,False,False,False,True,False
128,20076,False,False,False,False,True,False
11931,344342,False,False,False,False,True,False
9383,388463,False,False,False,False,True,False
...,...,...,...,...,...,...,...
6642,139238,False,False,False,False,True,True
3391,121185,False,False,False,False,True,True
21403,193446,False,False,False,False,True,True
7332,244119,False,False,False,False,True,True


In [39]:
os_gen=users_diseases[~ users_diseases['users'].isin(cancer_dataset['users'])& ~ users_diseases['users'].isin(diabetes_dataset['users'])& ~ users_diseases['users'].isin(heart_dataset['users'])& ~ users_diseases['users'].isin(ob_dataset['users'])& ~ users_diseases['users'].isin(den_dataset['users'])]
os_gen['Osteoporosis']=True
os_dataset=os_gen
os_dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
1,1773,False,False,False,False,False,True
3,2625,False,False,False,False,False,True
8,5523,False,False,False,False,False,True
9,42189,False,False,False,False,False,True
25,9622,False,False,False,False,False,True
...,...,...,...,...,...,...,...
25057,2002144249,False,False,False,False,False,True
25058,1800042015,False,False,False,False,False,True
25066,2002254807,False,False,False,False,False,True
25069,2001993143,False,False,False,False,False,True


In [40]:
dataset=pd.concat([cancer_dataset,diabetes_dataset,heart_dataset,ob_dataset,den_dataset,os_dataset,healthy_users]).drop_duplicates()
dataset

,users,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
17639,575940,True,False,False,False,False,False
7273,232086,True,False,False,False,False,False
7382,291378,True,False,False,False,False,False
20708,1028637,True,False,False,False,False,False
4054,118268,True,False,False,False,False,False
...,...,...,...,...,...,...,...
15856,847128,False,False,False,False,False,False
16876,401482,False,False,False,False,False,False
12261,393186,False,False,False,False,False,False
14881,539015,False,False,False,False,False,False


In [41]:
len(unique_users)

25076

In [42]:
dataset.to_csv("diseases_dataset.csv", 
                  index = None,
                  header=True)